In [1]:
from models.CNN_multitaper import ConvNetMultitaper
from models.LSTM_Conv import LSTM_Conv
from models.CNN_advanced import CNN_Advanced_model

from pre_processing import multitaper
from data_loader.data_loaders import MultiTaperSet, RawDataSet, FFT_Raw_DataSet
from trainer.trainer import Trainer

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import pandas as pd
import seaborn as sns


## Prepare Cuda


In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(device)

cpu


## Create MultiTapers

In [9]:
multitaper.compute_tapers()

load data
computing tapers


100%|██████████| 60/60 [00:08<00:00,  7.06it/s]


saving mutlitaper to: ./data/pre_processed_data/Multitaper_position_test


100%|██████████| 150/150 [00:29<00:00,  5.14it/s]


saving mutlitaper to: ./data/pre_processed_data/Multitaper_position_train


100%|██████████| 60/60 [02:34<00:00,  2.57s/it]


saving mutlitaper to: ./data/pre_processed_data/Multitaper_eeg_test


100%|██████████| 150/150 [07:30<00:00,  3.00s/it]


saving mutlitaper to: ./data/pre_processed_data/Multitaper_eeg_train


()

## Train basic CNN network (Timothée à faire)

## Training an advanced CNN network

In [3]:
from scipy import fftpack

data_path_train = './data/raw_data/X_train.h5'
data_path_test = './data/raw_data/X_test.h5'

target_path = './data/raw_data/y_train.csv'



raw_train_set = FFT_Raw_DataSet(device=device,
                                 data_path = data_path_train,
                                 target_path = target_path)

raw_test_set = FFT_Raw_DataSet(device=device,
                                 data_path = data_path_test,
                                 target_path = target_path)






1
2
4
5
6
over


/home/jupyter/deep_learning_project/data_loader/data_loaders.py:138: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  eeg_1 = torch.tensor(f['eeg_1'])


1
2
4
5
6
over


In [6]:
train_ratio = 0.9
# split train and validation set
data_set = raw_train_set
train_len = int(len(data_set)*train_ratio)
train_set, validation_set = torch.utils.data.random_split(data_set, [train_len, len(data_set) - train_len])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
validationloader = torch.utils.data.DataLoader(validation_set, batch_size=len(validation_set), shuffle=True)

In [24]:
#raw_feat, fft_feat, raw_pos_feat, fft_pos_feat = raw_train_set.feature_shape()
num_classes = 5
raw_feat, fft_feat, raw_pos_feat, fft_pos_feat = 5,5,3,3

In [25]:
CNN_Advanced_model = CNN_Advanced(raw_feat, fft_feat, raw_pos_feat, fft_pos_feat, num_classes, 0.5).to(device)


In [26]:
optimizer = torch.optim.Adam(CNN_Advanced_model.parameters())
trainer = Trainer(CNN_Advanced_model,
                  nn.CrossEntropyLoss(),
                  optimizer,
                  trainloader,
                  device,
                  valid_data_loader = validationloader,
                  class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]))




In [27]:
loss_list = []
accuracy_list = []
for epoch in range(0, 25):
    loss, accuracy = trainer.train_epoch()
    loss_list.append(loss)
    accuracy_list.append(accuracy)

100%|██████████| 211/211 [00:31<00:00,  6.63it/s]


average train loss:  1.3923545053219908
validation loss:  1.3547606468200684
validation accuracy:  0.45266666666666666


100%|██████████| 211/211 [00:31<00:00,  6.79it/s]


average train loss:  1.2895543925570085
validation loss:  1.2873822450637817
validation accuracy:  0.514


 76%|███████▌  | 160/211 [00:23<00:07,  6.76it/s]


KeyboardInterrupt: 

## Train CNN + Multitaper Model

In [10]:
CNN_taper_model = ConvNetMultitaper()
CNN_taper_model = CNN_taper_model.to(device)

In [11]:
multitaper_train_set = MultiTaperSet(device = device)
multitaper_test_set = MultiTaperSet(device=device, features_eeg_path = './data/pre_processed_data/Multitaper_eeg_test.npy', features_position_path = './data/pre_processed_data/Multitaper_position_test.npy')

In [12]:
train_ratio = 0.9
# split train and validation set
data_set = multitaper_train_set
train_len = int(len(data_set)*train_ratio)
train_set, validation_set = torch.utils.data.random_split(data_set, [train_len, len(data_set) - train_len])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
validationloader = torch.utils.data.DataLoader(validation_set, batch_size=len(validation_set), shuffle=True)

In [13]:
optimizer = torch.optim.Adam(CNN_taper_model.parameters())
trainer = Trainer(CNN_taper_model, nn.CrossEntropyLoss(), optimizer,trainloader, device, valid_data_loader = validationloader, class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]))

/home/jupyter/deep_learning_project/trainer/trainer.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.double)


In [14]:
loss_list = []
accuracy_list = []
for epoch in range(0, 25):
    loss, accuracy = trainer.train_epoch()
    loss_list.append(loss)
    accuracy_list.append(accuracy)

 89%|████████▉ | 188/211 [01:56<00:14,  1.62it/s]


KeyboardInterrupt: 

## Train CNN + LSTM

In [3]:
data_path_train = './data/raw_data/X_train.h5'
data_path_test = './data/raw_data/X_test.h5'

target_path = './data/raw_data/y_train.csv'



raw_train_set = RawDataSet(device=device,
                                 data_path = data_path_train,
                                 target_path = target_path)

raw_test_set = RawDataSet(device=device,
                                 data_path = data_path_test,
                                 target_path = target_path)





/home/jupyter/deep_learning_project/data_loader/data_loaders.py:53: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  eeg_1 = torch.tensor(f['eeg_1'])


In [4]:
train_ratio = 0.9
# split train and validation set
data_set = raw_train_set
train_len = int(len(data_set)*train_ratio)
train_set, validation_set = torch.utils.data.random_split(data_set, [train_len, len(data_set) - train_len])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
validationloader = torch.utils.data.DataLoader(validation_set, batch_size=len(validation_set), shuffle=True)

In [5]:
raw_feat = raw_train_set.feature_shape()
num_classes = 5


In [6]:
LSTM_Conv_model = LSTM_Conv(raw_feat, num_classes)
LSTM_Conv_model = LSTM_Conv_model.to(device)

In [8]:
optimizer = torch.optim.Adam(LSTM_Conv_model.parameters())

trainer = Trainer(LSTM_Conv_model,
                  nn.CrossEntropyLoss(),
                  optimizer,
                  trainloader,
                  device,
                  valid_data_loader = validationloader,
                  class_weights=torch.Tensor([8.081897,22.222222, 2.756846, 3.765060, 4.927727]), 
                  requires_softmax = True)


/home/jupyter/deep_learning_project/trainer/trainer.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.double)


In [9]:
loss_list = []
accuracy_list = []
for epoch in range(0, 25):
    loss, accuracy = trainer.train_epoch()
    loss_list.append(loss)
    accuracy_list.append(accuracy)

100%|██████████| 211/211 [01:50<00:00,  1.90it/s]


hello requires softmax
average train loss:  1.4462249663203814
validation loss:  1.4424412250518799
validation accuracy:  0.356


 72%|███████▏  | 152/211 [01:15<00:29,  2.01it/s]


KeyboardInterrupt: 